## Read database in

In [2]:
import pandas as pd
import sqlite3


#proof of concept - only has data from 9/7/19-9/21-19
conn = sqlite3.connect('mta_scrape1.db')
query = "SELECT *, DATE(DATETIME) AS DATE from entries where DESC IN ('REGULAR','RECOVR AUD')"

df = pd.read_sql_query(query,conn)

df.head()



,id,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,TIME,DESC,ENTRIES,EXITS,DATE
0,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 00:00:00,00:00:00,REGULAR,7198818,2438323,2019-09-14
1,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 04:00:00,04:00:00,REGULAR,7198834,2438325,2019-09-14
2,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 08:00:00,08:00:00,REGULAR,7198847,2438354,2019-09-14
3,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 12:00:00,12:00:00,REGULAR,7198929,2438428,2019-09-14
4,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 16:00:00,16:00:00,REGULAR,7199125,2438483,2019-09-14


### Calculate additional columns

In [3]:
#create turnstile index column
df['TURNSTILE_INDEX'] = df['CA'] + '-' +  df['UNIT'] + '-' + df['SCP'] + '-' + df['STATION']

df.head()

,id,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,TIME,DESC,ENTRIES,EXITS,DATE,TURNSTILE_INDEX
0,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 00:00:00,00:00:00,REGULAR,7198818,2438323,2019-09-14,A002-R051-02-00-00-59 ST
1,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 04:00:00,04:00:00,REGULAR,7198834,2438325,2019-09-14,A002-R051-02-00-00-59 ST
2,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 08:00:00,08:00:00,REGULAR,7198847,2438354,2019-09-14,A002-R051-02-00-00-59 ST
3,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 12:00:00,12:00:00,REGULAR,7198929,2438428,2019-09-14,A002-R051-02-00-00-59 ST
4,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 16:00:00,16:00:00,REGULAR,7199125,2438483,2019-09-14,A002-R051-02-00-00-59 ST


In [4]:
# Calculate ENTRIES_DIFF and EXITS_DIFF for each turnstile by derivative. 

turnstile_groupby = df.groupby(['CA','UNIT','SCP','STATION'])
for diff_field in ('ENTRIES','EXITS'):
    df[diff_field + '_DIFF'] = turnstile_groupby[diff_field].transform(pd.Series.diff)
    
df['FLOW_SUM'] = df['ENTRIES_DIFF'] + df['EXITS_DIFF']
    
df.head(15)

,id,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,TIME,DESC,ENTRIES,EXITS,DATE,TURNSTILE_INDEX,ENTRIES_DIFF,EXITS_DIFF,FLOW_SUM
0,1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 00:00:00,00:00:00,REGULAR,7198818,2438323,2019-09-14,A002-R051-02-00-00-59 ST,NaN,NaN,NaN
1,2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 04:00:00,04:00:00,REGULAR,7198834,2438325,2019-09-14,A002-R051-02-00-00-59 ST,16.0,2.0,18.0
2,3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 08:00:00,08:00:00,REGULAR,7198847,2438354,2019-09-14,A002-R051-02-00-00-59 ST,13.0,29.0,42.0
3,4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 12:00:00,12:00:00,REGULAR,7198929,2438428,2019-09-14,A002-R051-02-00-00-59 ST,82.0,74.0,156.0
4,5,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 16:00:00,16:00:00,REGULAR,7199125,2438483,2019-09-14,A002-R051-02-00-00-59 ST,196.0,55.0,251.0
5,6,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-14 20:00:00,20:00:00,REGULAR,7199405,2438527,2019-09-14,A002-R051-02-00-00-59 ST,280.0,44.0,324.0
6,7,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-15 00:00:00,00:00:00,REGULAR,7199547,2438545,2019-09-15,A002-R051-02-00-00-59 ST,142.0,18.0,160.0
7,8,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-15 04:00:00,04:00:00,REGULAR,7199567,2438547,2019-09-15,A002-R051-02-00-00-59 ST,20.0,2.0,22.0
8,9,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-15 08:00:00,08:00:00,REGULAR,7199579,2438564,2019-09-15,A002-R051-02-00-00-59 ST,12.0,17.0,29.0
9,10,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-09-15 12:00:00,12:00:00,REGULAR,7199648,2438630,2019-09-15,A002-R051-02-00-00-59 ST,69.0,66.0,135.0


In [ ]:
#create a new column converting date to day of week





### Filtering outlier values

In [6]:
# dropping NA values
df = df.dropna()


In [ ]:
#filter individual turnstile values using non-negatives and some arbitrarily large number.
min_flow = 0 #can't have negative flow
max_flow = 1_000_000 #filter billion+ entry records


df2 = df[(df['ENTRIES_DIFF'] > min_flow) & (df['ENTRIES_DIFF'] < max_flow) & (df['EXITS_DIFF'] > min_flow) & (df['EXITS_DIFF'] < max_flow)]

df2.head()

### Summarize turnstile info up to station level

In [8]:
#Group total flow ( Entries and exits) up to station level
#display top 20 stations

df2.groupby('STATION')['FLOW_SUM'].sum().sort_values(ascending=False).head(20)


STATION
34 ST-PENN STA     5707875.0
GRD CNTRL-42 ST    4939292.0
34 ST-HERALD SQ    4051054.0
14 ST-UNION SQ     3601842.0
23 ST              3567264.0
TIMES SQ-42 ST     3442144.0
FULTON ST          3384545.0
42 ST-PORT AUTH    3058183.0
86 ST              2964033.0
125 ST             2651975.0
CANAL ST           2477444.0
59 ST COLUMBUS     2469266.0
BEDFORD PK BLVD    2393226.0
14 ST              2211105.0
FLUSHING-MAIN      2063868.0
47-50 STS ROCK     2024535.0
96 ST              2017107.0
59 ST              1898803.0
CHAMBERS ST        1887858.0
PATH NEW WTC       1758409.0
Name: FLOW_SUM, dtype: float64

### Day of week summaries

In [ ]:
### Summarize turnstile data to day of week summary



### Other

In [9]:
#create summary of # of unique turnstiles/station
n_turnstiles = df.groupby('STATION')['TURNSTILE_INDEX'].nunique().sort_values(ascending=False)
                    
#top 10 stations by number of turnstiles    
n_turnstiles.head(10)

STATION
34 ST-PENN STA     100
FULTON ST           96
23 ST               73
GRD CNTRL-42 ST     69
86 ST               63
CANAL ST            58
CHAMBERS ST         54
34 ST-HERALD SQ     54
59 ST               52
42 ST-PORT AUTH     51
Name: TURNSTILE_INDEX, dtype: int64